In [ ]:
import gym

In [ ]:
import numpy as np
from keras.layers import Input, Conv2D, Dense, MaxPooling2D , Flatten
from keras.models import Model, Sequential, load_model
from collections import deque
from keras.optimizers import Adam
import keras

In [ ]:
env = gym.make('SpaceInvaders-v0')

In [ ]:
env.reset()
env.render()

In [ ]:
ac =env.action_space.sample()
o,r,d,i = env.step(1)
env.render()

In [ ]:

dqn = DQN(( 210 , 160, 3), 6)
dqn.policy_net.summary()

In [ ]:
stato = o.reshape((1,210,160,3))

In [ ]:
z=dqn.policy_net.predict(stato)

In [ ]:
z

In [ ]:
for t in range(1000):
    env.render()
    ac =env.action_space.sample()
    o,r,d,i = env.step(ac)
    if d:
        env.reset()
env.close()

In [ ]:
# con la funzione action_space riusciamo a trovare lo spazio delle azioni 
print (env.action_space)

In [ ]:
# con la funzione observation_space, invece, troviamo lo spazio delle osservazioni, in questo caso le dimensioni in pixel della
# nostra simulazione
print (env.observation_space)

In [ ]:
print (env.action_space)

In [ ]:
class DQN:
    def __init__(self, input_shape, output_shape, discount=0.99, update_target_every=10, memory_size=2000):
        self.input_shape=input_shape
        self.output_shape=output_shape
        self.discount=discount
        self.update_target_every=update_target_every
        self.policy_net=self.create_model()
        self.memory=deque(maxlen=memory_size)
        self.target_counter=0 
    
    def create_model(self):
        model=Sequential()
        model.add(Conv2D(input_shape=self.input_shape, filters=32, kernel_size=(7,7), strides=(5,5), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(Conv2D(filters=16, kernel_size=(5,5), strides=(4,4), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(Conv2D(filters=8, kernel_size=(3,3), strides=(2,2), padding="valid", 
                        activation="relu", use_bias=True,))
        model.add(Flatten())
        model.add(Dense(128, activation="relu"))
        model.add(Dense(self.output_shape, activation="softmax"))
        adm=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
        model.compile(loss="mse", optimizer=adm, metrics=["accuracy"])
        return model
    
    def get_action (self,state):
        action_prob= self.policy_net.predict(state)
        return action_prob
        
       
        